In [1]:
import lava
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from mod_one_exchange import mod_one_exchange_approximation
import graph_col_dimacs_reader as reader
import goemans_williamson_sdp as gw_sdp
import scipy
import matplotlib
import re
import time

In [2]:
G,adj,nnodes = reader.dimacs_reader("test_graphs/myciel6.col")
colors,cutsize = mod_one_exchange_approximation(G)
# Solve the SDP relaxation of the graph coloring problem
colors,cutsize = gw_sdp.goemans_williamson_max_cut(G)
# pos = nx.circular_layout(G)
# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111)
# nx.draw(G, pos, with_labels=True, edge_color='k', node_size=300, font_size=10, font_color='w', ax = ax)

# plt.show()

In [13]:
def random_cut(G):
    colors = np.random.randint(2, size = G.number_of_nodes())
    cutsize = 0
    for u,v in G.edges():
        if colors[u] != colors[v]:
            cutsize += 1
    return colors, cutsize

In [3]:
# Make a graph to mat converter
# convert adj to .mat file
def graph_to_mat(G, path = 'test_graphs/default2.mat'):
    # get the adjancency matrix of the graph
    adj = nx.adjacency_matrix(G).todense()
    adj = np.asarray(adj, dtype=np.int32)
    scipy.io.savemat(path, {'M': adj})   # M for matrix, to stay consistent with the .mat files we already have
    return


In [4]:
# Lets make mat to graph converter
def mat_to_graph(path):
    mat = scipy.io.loadmat(path)
    G = nx.from_numpy_matrix(mat['M'])
    return G


In [5]:
!python GraphColoringTest.py queen77 0.01 > temp_output7.txt

c:\Users\91702\Desktop\Sem_9\DDP\Maxcut\.venv\lib\site-packages\lava\magma\compiler\compiler_graphs.py:895: UserWarning: Cannot import module '<module 'graphcoloringmaxcut' from 'c:\\Users\\91702\\Desktop\\Sem_9\\DDP\\Maxcut\\graphcoloringmaxcut.py'>' when searching ProcessModels for Process 'GraphColorUpdate'.
  warnings.warn(
c:\Users\91702\Desktop\Sem_9\DDP\Maxcut\.venv\lib\site-packages\lava\magma\compiler\compiler_graphs.py:895: UserWarning: Cannot import module '<module 'graphcoloringmaxcut' from 'c:\\Users\\91702\\Desktop\\Sem_9\\DDP\\Maxcut\\graphcoloringmaxcut.py'>' when searching ProcessModels for Process 'OScillatoryNeuron'.
  warnings.warn(


In [6]:
G = mat_to_graph('test_graphs/queen77.mat')
def extract_chromatic_info(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    
    # Extract minimum order-based chromatic number using regex
    min_chromatic_number = int(re.search(r"minimum order-based chromatic number during simulation:  (\d+)", text).group(1))
    
    # Extract color blocks at minimum chromatic number using regex
    color_blocks_str = re.search(r"color blocks at minimum chromatic number:  (\[\[.*\]\])", text).group(1)
    color_blocks = eval(color_blocks_str)  # When you pass the string representation of a list of lists to eval(), 
    # it parses the string and converts it into actual Python objects (in this case, a list of lists of integers).
    
    return min_chromatic_number, color_blocks

min_chromatic_number_without_maxcut, color_blocks = extract_chromatic_info('temp_output7.txt')

colors = np.zeros(G.number_of_nodes())
for i, block in enumerate(color_blocks):
    for node in block:
        colors[node] = i

# check if the coloring is valid
valid_coloring = True
for edges in G.edges:
    if colors[edges[0]] == colors[edges[1]]:
        valid_coloring = False
        print("Invalid coloring")
        print(edges)
if valid_coloring:
    print("Valid coloring")

print(min_chromatic_number_without_maxcut)
print(color_blocks)


Valid coloring
17
[[14, 29], [8, 4], [7, 5, 27], [41, 16, 46, 12], [36, 19], [38, 18, 48], [31, 21], [45, 28, 2], [23, 13], [35, 1, 32], [33, 11, 15], [22, 6, 40], [10, 20, 0, 37], [24, 9, 47], [39, 17, 42], [30, 43, 26], [34, 25, 3, 44]]


In [7]:
def get_subgraphs(G, maxcut_colors):
    # maxcut_colors contain 0s and 1s
    # 0s are the nodes in the first set
    # 1s are the nodes in the second set
    # create two maps, one for each set
    # We proceed in ascending order over the original graph and assign the nodes to the sets
    # according to the maxcut_colors
    first_set = []
    second_set = []
    for i in range(len(maxcut_colors)):
        if maxcut_colors[i] == 0:
            first_set.append(i)
        else:
            second_set.append(i)

    # create the adjacency matrix for the subgraphs
    #
    adj = nx.adjacency_matrix(G).todense()
    adj = np.asarray(adj, dtype=np.int32)

    adj_sub1 = np.zeros((len(first_set), len(first_set)))
    adj_sub2 = np.zeros((len(second_set), len(second_set)))

    for i in range(len(first_set)):
        for j in range(len(first_set)):
            adj_sub1[i][j] = adj[first_set[i]][first_set[j]]

    for i in range(len(second_set)):
        for j in range(len(second_set)):
            adj_sub2[i][j] = adj[second_set[i]][second_set[j]]

    # create the subgraphs
    G_sub1 = nx.from_numpy_matrix(adj_sub1)
    G_sub2 = nx.from_numpy_matrix(adj_sub2)

    return G_sub1, G_sub2, first_set, second_set

In [14]:
# Now run mod one exchange on the graph
G = mat_to_graph('test_graphs/queen77.mat')
# maxcut_colors,cutsize = mod_one_exchange_approximation(G)
maxcut_colors,cutsize = random_cut(G)


# take the subgraph induced by part1 and part2 from their adjecency matrix
G_sub1, G_sub2, part1, part2 = get_subgraphs(G, maxcut_colors)

# Convert the subgraphs to .mat files
graph_to_mat(G_sub1, 'test_graphs/queen77_sub1.mat')
graph_to_mat(G_sub2, 'test_graphs/queen77_sub2.mat')

!python GraphColoringTest.py queen77_sub1 0.01 > temp_output_sub1.txt
!python GraphColoringTest.py queen77_sub2 0.01 > temp_output_sub2.txt

C:\Users\91702\AppData\Local\Temp\ipykernel_21328\3790719680.py:18: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G).todense()
C:\Users\91702\AppData\Local\Temp\ipykernel_21328\496379554.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G).todense()
c:\Users\91702\Desktop\Sem_9\DDP\Maxcut\.venv\lib\site-packages\lava\magma\compiler\compiler_graphs.py:895: UserWarning: Cannot import module '<module 'graphcoloringmaxcut' from 'c:\\Users\\91702\\Desktop\\Sem_9\\DDP\\Maxcut\\graphcoloringmaxcut.py'>' when searching ProcessModels for Process 'GraphColorUpdate'.
  warnings.warn(
c:\Users\91702\Desktop\Sem_9\DDP\Maxcut\.venv\lib\site-packages\lava\magma\compiler\compiler_graphs.py:895: UserWarning: Cannot import module '<module 'graphcoloringmaxcut' from 'c:\\Users\\91702\\Desktop\\Sem_9\\DDP\\Maxcut\\graphcoloringmaxcut.py'>

In [39]:
# extract chromatic info
min_chromatic_number1, color_blocks1 = extract_chromatic_info(file_path = 'temp_output_sub1.txt')
min_chromatic_number2, color_blocks2 = extract_chromatic_info(file_path = 'temp_output_sub2.txt')

print(min_chromatic_number1)
print(min_chromatic_number2)

colors1 = np.zeros(G_sub1.number_of_nodes())
colors2 = np.zeros(G_sub2.number_of_nodes())
color_index = 0
for i,block in enumerate(color_blocks1):
    for node in block:
        colors1[node] = color_index
    color_index += 1
for i,block in enumerate(color_blocks2):
    for node in block:
        colors2[node] = color_index
    color_index += 1

# check if the coloring is valid
valid_coloring1 = True
valid_coloring2 = True
G_sub1 = mat_to_graph('test_graphs/queen77_sub1.mat')
G_sub2 = mat_to_graph('test_graphs/queen77_sub2.mat')
for edges in G_sub1.edges:
    if colors1[edges[0]] == colors1[edges[1]]:
        valid_coloring1 = False
        print("Invalid coloring for subgraph 1")
        print(edges)
for edges in G_sub2.edges:
    if colors2[edges[0]] == colors2[edges[1]]:
        valid_coloring2 = False
        print("Invalid coloring for subgraph 2")
        print(edges)

if(valid_coloring1):
    print("Valid coloring for subgraph 1")
if(valid_coloring2):
    print("Valid coloring for subgraph 2")
    
# combine the two colorings
# Using two pointers approach
i = 0
j = 0
k = 0
merged_colors = np.zeros(G.number_of_nodes())
while i < G_sub1.number_of_nodes() and j < G_sub2.number_of_nodes():
    if maxcut_colors[k] == 0:
        merged_colors[part1[i]] = colors1[i]
        i += 1
    else:
        merged_colors[part2[j]] = colors2[j]
        j += 1
    k += 1
while i < G_sub1.number_of_nodes():
    merged_colors[part1[i]] = colors1[i]
    i += 1
    k += 1
while j < G_sub2.number_of_nodes():
    merged_colors[part2[j]] = colors2[j]
    j += 1
    k += 1

# check if the coloring is valid
valid_coloring = True
for edges in G.edges:
    if merged_colors[edges[0]] == merged_colors[edges[1]]:
        valid_coloring = False
        print("Invalid coloring for full Graph")
        print(edges)
if valid_coloring:
    print("Valid coloring for full Graph")






9
6
Valid coloring for subgraph 1
Valid coloring for subgraph 2
Valid coloring for full Graph


In [16]:
# Construct a map from of indices in subgraph to indices in the original graph
sub1_to_original = {}
sub2_to_original = {}
for i in range(len(part1)):
    sub1_to_original[i] = part1[i]
for i in range(len(part2)):
    sub2_to_original[i] = part2[i]

# check if the coloring is valid
valid_coloring1 = True
valid_coloring2 = True

for edges in G_sub1.edges:
    if colors1[edges[0]] == colors1[edges[1]]:
        valid_coloring1 = False
        print("Invalid coloring for subgraph 1")
        print(edges)
        # get the corresponding nodes in the original graph
        print(sub1_to_original[edges[0]], sub1_to_original[edges[1]])
for edges in G_sub2.edges:
    if colors2[edges[0]] == colors2[edges[1]]:
        valid_coloring2 = False
        print("Invalid coloring for subgraph 2")
        print(edges)
        # get the corresponding nodes in the original graph
        print(sub2_to_original[edges[0]], sub2_to_original[edges[1]])

if(valid_coloring1):
    print("Valid coloring for subgraph 1")
if(valid_coloring2):
    print("Valid coloring for subgraph 2")

    

Valid coloring for subgraph 1
Valid coloring for subgraph 2


In [17]:
# Check if the coloring is valid for merged graph
valid_coloring = True
for edges in G.edges:
    if merged_colors[edges[0]] == merged_colors[edges[1]]:
        valid_coloring = False
        print("Invalid coloring")
        print(edges)
if valid_coloring:
    print("Valid coloring")

Valid coloring


In [29]:
num_colors = min_chromatic_number1 + min_chromatic_number2
# num_colors*num_colors matrix to denote if th ecolors are connected or not
color_adj = np.zeros((num_colors,num_colors))
for edges in G.edges:
    color_adj[int(merged_colors[edges[0]])][int(merged_colors[edges[1]])] = 1
    color_adj[int(merged_colors[edges[1]])][int(merged_colors[edges[0]])] = 1

print(color_adj)
temp_check = np.ones((num_colors,num_colors)) - np.eye(num_colors)
if np.array_equal(color_adj, temp_check):
    print("Irreducible")
else:
    print("Reducible")
    


[[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]]
Reducible


In [30]:
# Create a graph from color adj
color_graph = nx.from_numpy_matrix(color_adj)
graph_to_mat(color_graph, 'test_graphs/color_graph.mat')



C:\Users\91702\AppData\Local\Temp\ipykernel_21328\496379554.py:5: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj = nx.adjacency_matrix(G).todense()


In [31]:
!python GraphColoringTest.py color_graph 0.01 > temp_color_output.txt

c:\Users\91702\Desktop\Sem_9\DDP\Maxcut\.venv\lib\site-packages\lava\magma\compiler\compiler_graphs.py:895: UserWarning: Cannot import module '<module 'graphcoloringmaxcut' from 'c:\\Users\\91702\\Desktop\\Sem_9\\DDP\\Maxcut\\graphcoloringmaxcut.py'>' when searching ProcessModels for Process 'GraphColorUpdate'.
  warnings.warn(
c:\Users\91702\Desktop\Sem_9\DDP\Maxcut\.venv\lib\site-packages\lava\magma\compiler\compiler_graphs.py:895: UserWarning: Cannot import module '<module 'graphcoloringmaxcut' from 'c:\\Users\\91702\\Desktop\\Sem_9\\DDP\\Maxcut\\graphcoloringmaxcut.py'>' when searching ProcessModels for Process 'OScillatoryNeuron'.
  warnings.warn(


In [34]:
# Find chromatic number using brute force
def min_chromatic_number(G):
    # Use networkx's greedy_color algorithm to assign colors
    color_assignment = nx.coloring.greedy_color(G, strategy="largest_first")
    
    # The chromatic number is the maximum color index used + 1
    chromatic_number = max(color_assignment.values()) + 1
    
    # Returning chromatic number and color assignment for nodes
    return chromatic_number, color_assignment

min_colors, _ = min_chromatic_number(color_graph)
print(min_colors)



14


In [37]:
min_colors


14

In [36]:
_

{0: 0,
 1: 1,
 3: 2,
 4: 3,
 5: 4,
 7: 5,
 8: 6,
 10: 7,
 12: 8,
 13: 9,
 6: 10,
 9: 11,
 11: 12,
 14: 13,
 2: 10}